<a href="https://colab.research.google.com/github/hashirmohammad/JigsawCommentClassification/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toxic Comment Classification — XGBoost Model
This notebook trains and evaluates an XGBoost-based multi-label classifier
for the Jigsaw Toxic Comment dataset. Logistic Regression was our baseline model;
here we explore a more powerful tree-based approach

In [ ]:
pip install iterative-stratification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
TRAIN_PATH = "/content/drive/MyDrive/MLProject/data/train.csv"
TEST_PATH = "/content/drive/MyDrive/MLProject/data/test.csv"
TEST_LABELS_PATH = "/content/drive/MyDrive/MLProject/data/test_labels.csv"

df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']


In [ ]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/MLProject')
from src.preprocess import clean_text
df["comment_text"] = df["comment_text"].apply(clean_text)
test_df["comment_text"] = test_df["comment_text"].apply(clean_text)


In [ ]:
tfidf = TfidfVectorizer(
    max_features=50000,
    ngram_range=(1,2),
    stop_words="english"
)

X = tfidf.fit_transform(df["comment_text"])
y = df[labels].values

X_test = tfidf.transform(test_df["comment_text"])


In [ ]:
mfold = MultilabelStratifiedKFold(n_splits=3, shuffle=True, random_state=42)

oof_preds = np.zeros((X.shape[0], len(labels)))
test_preds_folds = np.zeros((X_test.shape[0], len(labels)))


In [ ]:
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 6,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'tree_method': 'hist'
}
num_boost_round = 300

In [ ]:
for fold, (train_idx, val_idx) in enumerate(mfold.split(X, y)):
    print(f"\n=== FOLD {fold+1} ===")

    X_tr, X_va = X[train_idx], X[val_idx]
    y_tr, y_va = y[train_idx], y[val_idx]

    for i, lbl in enumerate(labels):
        print("Training label:", lbl)

        dtr = xgb.DMatrix(X_tr, label=y_tr[:, i])
        dva = xgb.DMatrix(X_va, label=y_va[:, i])

        bst = xgb.train(
          xgb_params,
          dtr,
          num_boost_round=num_boost_round,
          evals=[(dva, "valid")],
          callbacks=[xgb.callback.EarlyStopping(rounds=50, metric_name="logloss")],
          verbose_eval=False
        )


        # Store OOF predictions
        oof_preds[val_idx, i] = bst.predict(dva, iteration_range=(0, bst.best_iteration))

        # Store test predictions
        dtest = xgb.DMatrix(X_test)
        test_preds_folds[:, i] += bst.predict(dtest, iteration_range=(0, bst.best_iteration))

test_preds_folds /= mfold.n_splits



=== FOLD 1 ===
Training label: toxic
Training label: severe_toxic
Training label: obscene
Training label: threat
Training label: insult
Training label: identity_hate

=== FOLD 2 ===
Training label: toxic
Training label: severe_toxic
Training label: obscene
Training label: threat
Training label: insult
Training label: identity_hate

=== FOLD 3 ===
Training label: toxic
Training label: severe_toxic
Training label: obscene
Training label: threat
Training label: insult
Training label: identity_hate


In [ ]:
print("=== OOF Classification Report ===")
print(classification_report(y, (oof_preds > 0.2).astype(int), target_names=labels))


=== OOF Classification Report ===
               precision    recall  f1-score   support

        toxic       0.79      0.67      0.73     15294
 severe_toxic       0.40      0.56      0.46      1595
      obscene       0.80      0.80      0.80      8449
       threat       0.45      0.42      0.43       478
       insult       0.68      0.71      0.69      7877
identity_hate       0.46      0.46      0.46      1405

    micro avg       0.72      0.69      0.71     35098
    macro avg       0.60      0.60      0.60     35098
 weighted avg       0.73      0.69      0.71     35098
  samples avg       0.06      0.06      0.06     35098



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
test_labels = pd.read_csv(TEST_LABELS_PATH)
valid_idx = (test_labels[labels] != -1).all(axis=1)

y_test = test_labels.loc[valid_idx, labels].values
test_preds_valid = test_preds_folds[valid_idx]


In [ ]:
threshold = 0.2
test_preds_bin = (test_preds_valid > threshold).astype(int)


In [ ]:
print("=== XGBoost Test Set Report ===")
print(classification_report(y_test, test_preds_bin, target_names=labels))


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


=== XGBoost Test Set Report ===
               precision    recall  f1-score   support

        toxic       0.52      0.79      0.63      6090
 severe_toxic       0.24      0.63      0.34       367
      obscene       0.55      0.79      0.65      3691
       threat       0.30      0.58      0.40       211
       insult       0.50      0.72      0.59      3427
identity_hate       0.52      0.61      0.56       712

    micro avg       0.51      0.76      0.61     14498
    macro avg       0.44      0.69      0.53     14498
 weighted avg       0.51      0.76      0.61     14498
  samples avg       0.06      0.07      0.06     14498

